<a href="https://colab.research.google.com/github/tort-cam/ST-554-P1/blob/main/Task1/task01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Title: Task 1

Author: Bryan Sandor

# Initialization

The goal for this task is to use two methods to minimize the root mean square error (RMSE) for two types of regression:

- One with a horizontal line of best fit for the data, i.e., $y = c$ for some constant real value $c$.
- One with a line of best fit for the data, i.e., $y = \beta_0 + \beta_1 x$ where $x$ is a predictor variable, $y$ is a response variable, and $\beta_0$ and $\beta_1$ are constant real values.

The data being made available through the UCI website, a package must be installed before the appropriate file may be accessed and the necessary variables saved locally. Along with this we import standard packages for use later.

In [1]:
!pip install ucimlrepo

import pandas as pd
import math as math
import numpy as np

from ucimlrepo import fetch_ucirepo

# fetch dataset
air_quality = fetch_ucirepo(id=360)

# data (as pandas dataframes)
airdata = air_quality.data.features
y = air_quality.data.targets

# metadata
print(air_quality.metadata)

# variable information
print(air_quality.variables)

# look at a few of the observations
airdata

{'uci_id': 360, 'name': 'Air Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/360/air+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/360/data.csv', 'abstract': 'Contains the responses of a gas multisensor device deployed on the field in an Italian city. Hourly responses averages are recorded along with gas concentrations references from a certified analyzer. ', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 9358, 'num_features': 15, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C59K5F', 'creators': ['Saverio Vito'], 'intro_paper': {'ID': 420, 'type': 'NATIVE', 'title': 'On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario', 'authors': 

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


Now we delete any observations where `C6H6(GT)` or `PT08.S1(CO)` are -200, which represent missing values.

In [2]:
sum(airdata["C6H6(GT)"] == -200) # count number of observations to delete

366

In [3]:
for i in range(len(airdata)):
    if (airdata["C6H6(GT)"].loc[i] == -200):
        airdata.drop(i, inplace = True) # inplace updates dataframe actively

In [4]:
sum(airdata["C6H6(GT)"] == -200)

0

Notice the observations indicating missing values for `C6H6(GT)` must have also been missing values for `PT08.S1(CO)` since the number of observations indicating missing values for the latter is now also $0$. A `len` command also verifies $366$ observations have been removed from the dataframe.

In [5]:
sum(airdata["PT08.S1(CO)"] == -200)

0

In [6]:
len(airdata)

8991

# Grid Search (Brute Force)

In this subsection we use a brute-force search within a defined range of values to find the optimal constant/coefficient for our lines of best fit.

## Just `y`

Here, we use brute-force to search for the constant $c$ satisfying a line of best fit $y = c$. Note that the optimal solution is the mean value of set of data used.

We first create three functions:
1. A loss function, which calculates and returns the RMSE between all the observations and a fixed constant, `c`.

In [7]:
def loss(y = "C6H6(GT)", c = 0):

    """
    This function takes a column from the airdata data frame and a fixed value
     c and returns the root mean square error of all the observations in the
     desired column and c.
    """

    MSE = 0 # initialize mean square error (MSE)

    for i in range(len(airdata)): # calculate MSE
        MSE += ( airdata[y].iloc[i] - c ) ** 2

    MSE = MSE/len(airdata)

    RMSE = np.sqrt(MSE) # calculate root MSE (RMSE)

    return RMSE

2. A function creating a uniform "grid" (here the grid is one-dimensional) of values initializing and terminating at two given values and a fixed number of "steps".

In [8]:
def makegrid(y = "C6H6(GT)", a = airdata["C6H6(GT)"].quantile(0.25), \
               b = airdata["C6H6(GT)"].quantile(0.75), n = 1):

    """
    This function creates a list for a grid of values spanning from the first
     quartile to the third quartile with the step size determined by the number
      of steps in the span, n.
    """

    mesh = (b - a)/n
    grid = [x * mesh + a for x in range(n)]
    return grid

3. A function using the previous two to create and scour the grid, using those different values for `c` to find the smallest RMSE, returning both its value and the value for `c` where it occurs.

In [9]:
def gridsearch(y = "C6H6(GT)", a = airdata["C6H6(GT)"].quantile(0.25), \
               b = airdata["C6H6(GT)"].quantile(0.75), n = 1):
    """
    This function takes a column from the airdata, y, a lower and upper bound
     of values to begin and terminate search, and a desired number of steps, n,
     creates a grid and runs the loss function on each value within the grid to
     create a list. It then searches the list for the minimum root mean square
     error and the position within the grid where it occurs, returning both.
    """

    if n > 1000: # Program takes approximately 1 minute to run with this cap
        print("Please choose a smaller number of iterations.")
        return

    grid = makegrid(y = y, a = a, b = b, n = n) # create a grid with given parameters

    rmsegrid = [[grid[i], loss(y = y, c = grid[i])] for \
                i in range(n)] # list comprehension to compute the root mean square error for all the values within the grid

    minrmse = rmsegrid[0][1] # initializes the minimum RMSE
    gridref = rmsegrid[0][0] # initializes value of c achieving minimum RMSE

    for i in range(n): # finds the smallest RMSE and its location in the grid
        if rmsegrid[i][1] < minrmse:
            minrmse = rmsegrid[i][1]
            gridref = rmsegrid[i][0]

    print("The minimum RMSE is", round(minrmse, 3))
    print("The value for c producing the minimum RMSE is", round(gridref, 3))

    return

Running the function with a step size of 1000 yields the following:

In [10]:
gridsearch(n = 1000) # this takes approximately a minute to run

The minimum RMSE is 7.449
The value for c producing the minimum RMSE is 10.083


Note that the value of `c` yielding the minimum RMSE is the mean of the data, given below and approximately matching that found by the brute force function.

In [11]:
print(round(airdata["C6H6(GT)"].mean(), 3))

10.083


Similarly, we verify such a value for `c` produces an RMSE similar to that found by the function.

In [12]:
print(round( np.sqrt( \
                     np.mean( \
                      (airdata["C6H6(GT)"] - \
                       airdata["C6H6(GT)"].mean()) ** 2)), \
              3))

7.449


We repeat the process, this time on the variable `PT08.S1(CO)` instead to verify the process works independent of variable chosen.

In [13]:
gridsearch(y = "PT08.S1(CO)", a = airdata["PT08.S1(CO)"].quantile(0.25), \
           b = airdata["PT08.S1(CO)"].quantile(0.75), n = 1000)
# This takes about a minute to run

The minimum RMSE is 217.068
The value for c producing the minimum RMSE is 1099.876


In [14]:
print(round(airdata["PT08.S1(CO)"].mean(), 3))

1099.833


In [15]:
print(round( np.sqrt( \
                     np.mean( \
                      (airdata["PT08.S1(CO)"] - \
                       airdata["PT08.S1(CO)"].mean()) ** 2)), \
             3))

217.068


## Using `y` and another numeric variable `x`

We now mimic the previous section, using instead a linear function, $y = \beta_0 + \beta_1 x$, to attempt to minimize the RMSE.

In [16]:
def loss2d(x = "PT08.S1(CO)", y = "C6H6(GT)", beta0 = 0, beta1 = 0):

    """
    This function takes two columns from the airdata data frame, two parameters
     for the linear regression, beta0 and beta1, and returns the root mean
     square error of the differences between the expected and observed values.
    """

    MSE = 0 # initialize mean square error (MSE)

    for i in range(len(airdata)): # calculate MSE
        MSE += ( airdata[y].iloc[i] - \
                beta0 - beta1 * airdata[x].iloc[i] ) ** 2

    MSE = MSE/len(airdata)

    RMSE = np.sqrt(MSE) # calculate root MSE (RMSE)

    return RMSE

In [17]:
def makegrid2d(a = -24, b = -23, n = 1, c = 0, d = 1, m = 1):

    """
    This function creates a list for a 2d-grid of values spanning from the
    first endpoint to the second endpoint with the step size determined by the
    number of steps in the span, n and the third endpoint to the fourth
    endpoint with the step size determined by the number of steps in that span,
    m.
    """

    meshb0 = (b - a)/n
    meshb1 = (d - c)/m
    grid2d = [[0] * m for _ in range(n)]

    for i in range(n):
        for j in range(m):
            grid2d[i][j] = [ a + meshb0 * i, c + meshb1 * j ]

    return grid2d

In [18]:
def grid2dsearch(x = "PT08.S1(CO)", y = "C6H6(GT)", \
                 a = -24, b = -23, n = 1, \
                 c = 0, d = 1, m = 1):
    """
    This function takes a column from the airdata, y, a lower and upper bound
    of values to begin and terminate search, and a desired number of steps, n,
    creates a grid and runs the loss function on each value within the grid to
    create a list. It then searches the list for the minimum root mean square
    error and the position within the grid where it occurs, returning both.
    """

    grid2d = makegrid2d(a = a, b = b, n = n, c = c, d = d, m = m)
    # create a 2dgrid with given parameters

    minrmse = loss2d(x = x, y = y, \
                     beta0 = grid2d[0][0][0], \
                     beta1 = grid2d[0][0][1]) # initialize minimum RMSE value
    minb0 = grid2d[0][0][0] # initialize minimum RMSE beta-0
    minb1 = grid2d[0][0][1] # initialize minimum RMSE beta-1

    for i in range(n):
        for j in range(m):
            if loss2d(x = x, y = y, \
                      beta0 = grid2d[i][j][0], \
                      beta1 = grid2d[i][j][1]) < minrmse: # if the loss function returns a smaller RMSE, update the current smallest RMSE and the beta parameters for it
                        minrmse = loss2d(x = x, y = y, \
                                         beta0 = grid2d[i][j][0], \
                                         beta1 = grid2d[i][j][1])
                        minb0 = grid2d[i][j][0]
                        minb1 = grid2d[i][j][1]

    print("The minimum RMSE is", round(minrmse, 3))
    print("The value for beta-0 producing it is", round(minb0, 3))
    print("The value for beta-1 producing it is", round(minb1, 3))

    return

The search parameters are narrower than those specified in the directions to assist the written code in finding the correct values of $\beta_0$ and $\beta_1$ in a "timely fashion."

In [19]:
grid2dsearch(a = -24, b = -23, n = 10, c = 0, d = 1, m = 100)
# takes 3 minutes to run

The minimum RMSE is 3.491
The value for beta-0 producing it is -23.1
The value for beta-1 producing it is 0.03


Finally, we compare the results to the more efficient exact formula and native function from `numpy`:

In [20]:
y = airdata["C6H6(GT)"]
x = airdata["PT08.S1(CO)"]

b1hat = sum((x - x.mean())*(y - y.mean()))/sum((x - x.mean())**2)
b0hat = y.mean() - x.mean()*b1hat

print("The minimizing values for beta-0 and beta-1 are", \
      round(b0hat, 5), "and", \
      round(b1hat, 5), ", respectively")

meanSquaredError = ((b0hat + b1hat * x - y) ** 2).mean()

print("The minimum RMSE is", round(np.sqrt(meanSquaredError), 3))

The minimizing values for beta-0 and beta-1 are -23.27522 and 0.03033 , respectively
The minimum RMSE is 3.485


# Gradient Descent (Tangent Lines)

For this section, we use a more efficient algorithm to find the optimal values: one utilizing the slopes of tangent lines, i.e., the derivative, to minimize the RMSE.

## Just `y`

We may use the same root mean square error (the "loss") function from the grid search section. However, we will need to craft a new function that finds the difference quotient approximation to the derivative.

In [21]:
def diffquo(y = "C6H6(GT)", c = 0, delta = 1):

    """
    Calculates the difference quotient for the root mean square (loss) function
    given an initial guess for c and a delta value.
    """

    dq = (loss(y = y, c = c + delta) - loss(y = y, c = c)) / delta

    return dq

Now, using this function, we compute a linear approximation to the RMSE function at a guess value of `c`, a small change to `c`, $\Delta$, and a small step size to travel along the linear approximation towards the direction of the minimizing value, subject to a desired level of tolerance. Finally, to prevent a non-terminating code loop, we include a count of the number of cycles and break out of the code once it is reached.

In [22]:
def graddesc(y = "C6H6(GT)", c = 0, delta = 1, step = 1, tol = 1, maxiteration = 5000):

    """
    This function takes a given variable, a guess c, a small change (to c) of
     delta, a step size, given level of tolerance, and a maximum number for the
     loop used to determine the minimum RMSE and its corresponding value.
    """

    nextc = c - diffquo(y, c, delta) * step # compute first new c
    iteration = 1 # initialize the iteration count

    while abs(nextc - c) >= tol: # perform the loop while the differences in successive c values is "large"
        if iteration >= maxiteration: # break out of the loop once the maximum number of iterations is met and print the current values
            print("Iteration threshhold of", maxiteration, "cycles surpassed.")
            print("The current value of c is", round(nextc, 3), "and its corresponding RMSE is", round(loss(y, c = nextc), 3))
            return
        else: # otherwise, compute successive values of c
            c = nextc
            nextc = c - diffquo(y, c, delta) * step
            iteration += 1

    print("After", iteration, "iterations, The current value of c is", round(nextc, 3), "and its corresponding RMSE is", round(loss(y, c = nextc), 3))

    return

To test the iteration threshhold, we use a small value for `maxiteration`.

In [23]:
graddesc(c = 0, delta = 1, step = 1, tol = 0.001, maxiteration = 10)

Iteration threshhold of 10 cycles surpassed.
The current value of c is 6.321 and its corresponding RMSE is 8.346


Finally, we use the method of gradual descent to approximate the minimum RMSE and its corresponding value, recalling from the previous section the more efficiently computed values of $RMSE \approx 7.449$ and $c \approx 10.083$, respectively.

In [24]:
graddesc(delta = 0.001, step = 0.01, tol = 0.0001)
# this takes approximately 8.5 minutes to run

After 3945 iterations, The current value of c is 10.008 and its corresponding RMSE is 7.45


Finally, to test for generalization, we run the function on the variable `PT08.S1(CO)` with the recommended values from the notes. From before, the minimum $RMSE \approx 217.068$ and occurs when $c \approx 1099.883$.

In [26]:
graddesc(y = "PT08.S1(CO)", c = 1000, delta = 0.001, step = 0.1, tol = 0.0001, maxiteration = 10000)
# this takes 22.5 minutes to run

Iteration threshhold of 10000 cycles surpassed.
The current value of c is 1098.784 and its corresponding RMSE is 217.07


As a final note, the directions indicated to use $c = 1100$ as a starting value, but observe the speed of the convergence:

In [27]:
graddesc(y = "PT08.S1(CO)", c = 1100, delta = 0.001, step = 0.1, tol = 0.0001, maxiteration = 10000)

After 1 iterations, The current value of c is 1100.0 and its corresponding RMSE is 217.068


## Using `y` and another numeric variable `x`

We conclude this portion of the assignment by now applying the previous technique in two dimensions on the parameters $\beta_0$ and $\beta_1$. We begin with a modified difference quotient on the $\beta_0$ parameter.

In [28]:
def diffquob0(x = "PT08.S1(CO)", y = "C6H6(GT)", \
              beta0 = -20, beta1 = 0, delta0 = 1):

    """
    Calculates the difference quotient on beta-0 for the root mean square (loss) function given an initial guess for beta-0 and a delta-0 value.
    """

    dq = (loss2d(x, y, beta0 + delta0, beta1) - \
          loss2d(x, y, beta0, beta1)) / delta0

    return dq

Now we apply a similar difference quotient on the $\beta_1$ parameter.

In [29]:
def diffquob1(x = "PT08.S1(CO)", y = "C6H6(GT)", \
              beta0 = -20, beta1 = 0, delta1 = 1):

    """
    Calculates the difference quotient on beta-1 for the root mean square
    (loss) function given an initial guess for beta-1 and a delta-1 value.
    """

    dq = (loss2d(x, y, beta0, beta1 + delta1) - \
          loss2d(x, y, beta0, beta1)) / delta1

    return dq

Then we modify the previous gradient descent function to include both $\beta_0$ and $\beta_1$.

In [30]:
def graddesc2d(x = "PT08.S1(CO)", y = "C6H6(GT)", \
               beta0 = -20, beta1 = 0, delta0 = 1, delta1 = 1, \
               stepb0 = 1, stepb1 = 1, tol = 1, maxiteration = 5000):

    """
    This function performs the gradient descent method on beta-0 and beta-1 to
     minimize the RMSE.
    """
    # compute the first values of beta-0 and beta-1 and compare them, in distance, to the previous values
    nextb0 = beta0 - diffquob0(x, y, beta0, beta1, delta0) * stepb0
    nextb1 = beta1 - diffquob1(x, y, beta0, beta1, delta1) * stepb1
    change = np.sqrt( (nextb0 - beta0) ** 2 + (nextb1 - beta1) ** 2 )
    iteration = 1 # initialize the iteration count

    while change >= tol: # perform the loop while the differences in successive c values is "large"
        if iteration >= maxiteration: # break out of the loop once the maximum number of iterations is met and print the current values
            print("Iteration threshhold of", maxiteration, "cycles surpassed.")
            print("The current value of beta-0 is", round(nextb0, 3), ", beta-1 is", round(nextb1, 3), ", and the corresponding RMSE is", round(loss2d(x, y, nextb0, nextb1), 3))
            return
        else: # otherwise, compute successive values of beta-0 and beta-1
            beta0 = nextb0
            nextb0 = beta0 - diffquob0(x, y, beta0, beta1, delta0) * stepb0

            beta1 = nextb1
            nextb1 = beta1 - diffquob1(x, y, beta0, beta1, delta1) * stepb1

            change = np.sqrt( (nextb0 - beta0) ** 2 + (nextb1 - beta1) ** 2 )

            iteration += 1

    print("After", iteration, "iterations, the current value of beta-0 is", round(nextb0, 3), ", beta-1 is", round(nextb1, 3), ", and the corresponding RMSE is", round(loss2d(x, y, nextb0, nextb1), 3))

    return

Running the program with the given parameters from the notes results in the following output. Recall from the previous section the optimal values are $\beta_0 \approx -23.27522$ and $\beta_1 \approx 0.03033$, having $RMSE \approx 3.485$.

In [31]:
graddesc2d(beta0 = -23.2, beta1 = 0.03, delta0 = 0.005, delta1 = 0.005, stepb0 = 0.001, stepb1 = 0.00001, tol = 0.0001, maxiteration = 1000)
# this takes approximately 8 minutes to run

Iteration threshhold of 1000 cycles surpassed.
The current value of beta-0 is -23.01 , beta-1 is 0.032 , and the corresponding RMSE is 4.084


Finally, we craft a predictor function utilizing stated values of $\beta_0$ and $\beta_1$ on the predictor variable `PT08.S1(CO)` to estimate the response variable `C6H6(CO)`.

In [32]:
def response(predictor = 0, beta0 = -23.27522, beta1 = 0.03033):
    estimate = beta0 + beta1 * predictor
    print("The estimated response value for C6H6(GT) is", round(estimate, 3))
    return

In [33]:
response(946, beta0 = -23.01, beta1 = 0.032)

The estimated response value for C6H6(GT) is 7.262


For reference, within the data the values corresponding to a `PT08.S1(CO)` of 946 range from 2.1 to 8.8.

In [34]:
response(1075, beta0 = -23.01, beta1 = 0.032)

The estimated response value for C6H6(GT) is 11.39


For reference, within the data the values corresponding to a `PT08.S1(CO)` of 1075 range from 3.9 to 17.4.

In [35]:
response(1246, beta0 = -23.01, beta1 = 0.032)

The estimated response value for C6H6(GT) is 16.862


For reference, within the data the values corresponding to a `PT08.S1(CO)` of 1246 range from 8.9 to 17.